In [ ]:
import pandas as pd

In [ ]:
matches = pd.read_csv('Matches.csv')

matches.head()

/tmp/ipython-input-500440820.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  matches = pd.read_csv('Matches.csv')


,Division,MatchDate,MatchTime,HomeTeam,AwayTeam,HomeElo,AwayElo,Form3Home,Form5Home,Form3Away,...,MaxHome,MaxDraw,MaxAway,Over25,Under25,MaxOver25,MaxUnder25,HandiSize,HandiHome,HandiAway
0,F1,2000-07-28,NaN,Marseille,Troyes,1686.34,1586.57,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F1,2000-07-28,NaN,Paris SG,Strasbourg,1714.89,1642.51,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F2,2000-07-28,NaN,Wasquehal,Nancy,1465.08,1633.80,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F2,2000-07-29,NaN,Ajaccio,Le Mans,1470.87,1477.89,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F2,2000-07-29,NaN,Beauvais,Montpellier,1422.21,1606.00,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
matches['MatchDate'] = pd.to_datetime(matches['MatchDate'], errors='coerce')
matches = matches.sort_values(['MatchDate']).reset_index(drop=True)

matches.head()

,Division,MatchDate,MatchTime,HomeTeam,AwayTeam,HomeElo,AwayElo,Form3Home,Form5Home,Form3Away,...,MaxHome,MaxDraw,MaxAway,Over25,Under25,MaxOver25,MaxUnder25,HandiSize,HandiHome,HandiAway
0,F1,2000-07-28,NaN,Marseille,Troyes,1686.34,1586.57,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F1,2000-07-28,NaN,Paris SG,Strasbourg,1714.89,1642.51,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F2,2000-07-28,NaN,Wasquehal,Nancy,1465.08,1633.80,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F1,2000-07-29,NaN,Toulouse,Bastia,1551.65,1656.37,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F1,2000-07-29,NaN,Nantes,Lens,1664.36,1730.89,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def get_last5_stats(df, n=5):
    # Create a long format (team, opponent, goals_for, goals_against, result)
    home = df[['MatchDate', 'HomeTeam', 'AwayTeam', 'FTHome', 'FTAway', 'FTResult']].copy()
    home.rename(columns={
        'HomeTeam': 'Team',
        'AwayTeam': 'Opponent',
        'FTHome': 'GoalsFor',
        'FTAway': 'GoalsAgainst'
    }, inplace=True)
    home['IsHome'] = 1

    away = df[['MatchDate', 'AwayTeam', 'HomeTeam', 'FTAway', 'FTHome', 'FTResult']].copy()
    away.rename(columns={
        'AwayTeam': 'Team',
        'HomeTeam': 'Opponent',
        'FTAway': 'GoalsFor',
        'FTHome': 'GoalsAgainst'
    }, inplace=True)
    away['IsHome'] = 0

    long_df = pd.concat([home, away], ignore_index=True)

    # Compute points
    def get_points(row):
        if row['GoalsFor'] > row['GoalsAgainst']:
            return 3
        elif row['GoalsFor'] == row['GoalsAgainst']:
            return 1
        else:
            return 0

    long_df['Points'] = long_df.apply(get_points, axis=1)
    long_df['Win'] = (long_df['Points'] == 3).astype(int)
    long_df['Draw'] = (long_df['Points'] == 1).astype(int)
    long_df['Loss'] = (long_df['Points'] == 0).astype(int)

    # Sort by team/date for rolling
    long_df = long_df.sort_values(['Team', 'MatchDate'])

    # Rolling stats
    long_df['Last5_Points'] = long_df.groupby('Team')['Points'].transform(lambda x: x.shift().rolling(n, min_periods=1).sum())
    long_df['Last5_GoalsFor'] = long_df.groupby('Team')['GoalsFor'].transform(lambda x: x.shift().rolling(n, min_periods=1).sum())
    long_df['Last5_GoalsAgainst'] = long_df.groupby('Team')['GoalsAgainst'].transform(lambda x: x.shift().rolling(n, min_periods=1).sum())
    long_df['Last5_Wins'] = long_df.groupby('Team')['Win'].transform(lambda x: x.shift().rolling(n, min_periods=1).sum())
    long_df['Last5_Draws'] = long_df.groupby('Team')['Draw'].transform(lambda x: x.shift().rolling(n, min_periods=1).sum())
    long_df['Last5_Losses'] = long_df.groupby('Team')['Loss'].transform(lambda x: x.shift().rolling(n, min_periods=1).sum())

    return long_df

In [ ]:
long_stats = get_last5_stats(matches, n=5)

# Merge back to match-level (home/away)
home_stats = long_stats[long_stats['IsHome'] == 1].drop(columns=['Opponent','IsHome','Points','Win','Draw','Loss','GoalsFor','GoalsAgainst','FTResult'])
away_stats = long_stats[long_stats['IsHome'] == 0].drop(columns=['Opponent','IsHome','Points','Win','Draw','Loss','GoalsFor','GoalsAgainst','FTResult'])

home_stats = home_stats.add_prefix('Home_')
away_stats = away_stats.add_prefix('Away_')

# Merge into original matches
matches = matches.merge(home_stats, left_on=['HomeTeam','MatchDate'], right_on=['Home_Team','Home_MatchDate'])
matches = matches.merge(away_stats, left_on=['AwayTeam','MatchDate'], right_on=['Away_Team','Away_MatchDate'])

# Clean up duplicate columns
matches = matches.drop(columns=['Home_Team','Home_MatchDate','Away_Team','Away_MatchDate'])

# ✅ Now matches has last-5 stats for both teams
print(matches.head(20)[[
    'HomeTeam','AwayTeam',
    'Home_Last5_Points','Home_Last5_GoalsFor','Home_Last5_GoalsAgainst','Home_Last5_Wins','Home_Last5_Draws','Home_Last5_Losses',
    'Away_Last5_Points','Away_Last5_GoalsFor','Away_Last5_GoalsAgainst','Away_Last5_Wins','Away_Last5_Draws','Away_Last5_Losses'
]])

       HomeTeam     AwayTeam  Home_Last5_Points  Home_Last5_GoalsFor  \
0     Marseille       Troyes                NaN                  NaN   
1      Paris SG   Strasbourg                NaN                  NaN   
2     Wasquehal        Nancy                NaN                  NaN   
3      Toulouse       Bastia                NaN                  NaN   
4        Nantes         Lens                NaN                  NaN   
5          Lyon       Rennes                NaN                  NaN   
6         Lille       Monaco                NaN                  NaN   
7      Bordeaux         Metz                NaN                  NaN   
8       Auxerre        Sedan                NaN                  NaN   
9         Niort       Angers                NaN                  NaN   
10     Guingamp   St Etienne                NaN                  NaN   
11         Nice     Le Havre                NaN                  NaN   
12        Laval     Gueugnon                NaN                 

In [ ]:
output_file = "Matches_with_last5.csv"
matches.to_csv(output_file, index=False)

In [ ]:
print(matches.tail())

       Division  MatchDate MatchTime       HomeTeam   AwayTeam  HomeElo  \
228372       E0 2025-02-26  19:30:00      Tottenham   Man City  1785.53   
228373       E0 2025-02-26  19:30:00      Brentford    Everton  1738.05   
228374       E0 2025-02-26  19:30:00     Man United    Ipswich  1757.62   
228375       E0 2025-02-26  19:30:00  Nott'm Forest    Arsenal  1788.28   
228376       E0 2025-02-26  20:15:00      Liverpool  Newcastle  1995.58   

        AwayElo  Form3Home  Form5Home  Form3Away  ...  \
228372  1926.48        9.0        9.0        3.0  ...   
228373  1731.52        6.0        9.0        5.0  ...   
228374  1584.51        1.0        4.0        1.0  ...   
228375  1999.49        3.0        6.0        6.0  ...   
228376  1826.18        7.0       11.0        3.0  ...   

        Home_Last5_GoalsAgainst  Home_Last5_Wins  Home_Last5_Draws  \
228372                      6.0              3.0               0.0   
228373                      5.0              3.0               0.0